# Start Simulation

In [1]:
# env.clean(clean_all=True)

In [2]:
import random
import numpy as np
import os
import time
from PIL import Image

import torch
torch.__version__
device = torch.device("cuda")

In [3]:
import getpass
user = getpass.getuser()
print(user)

yizhou


In [4]:
usd_path = f"omniverse://localhost/Users/{user}/UVA/uva_desk.usd"

In [5]:
from omni.isaac.kit import SimulationApp    
simulation_app = SimulationApp({"headless": True, "open_usd": usd_path,  "livesync_usd": usd_path}) 

Starting kit application with the fillowing args:  ['/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps', '--/physics/cudaDevice=0', '--portable', '--no-window']
Passing the following args to the base kit application:  ['-f', '/home/yizhou/.local/share/jupyter/runtime/kernel-931771f0-d65f-

OmniAssetFileFormat


eline-1.0.2] startup
[1.959s] [ext: omni.hydra.scene_delegate-0.2.0] startup
[1.976s] [ext: omni.kit.audiodeviceenum-1.0.0] startup
[1.979s] [ext: omni.usd-1.5.3] startup
[2.112s] [ext: omni.kit.asset_converter-1.2.31] startup
[2.163s] [ext: omni.usd.schema.physx-0.0.0] startup
[2.207s] [ext: omni.usd.schema.isaac-0.2.0] startup
[2.278s] [ext: omni.usd.schema.forcefield-0.0.0] startup
[2.290s] [ext: omni.kvdb-0.0.0] startup
[2.295s] [ext: omni.usdphysics-1.4.15] startup
[2.298s] [ext: omni.graph.tools-1.4.0] startup
[2.331s] [ext: omni.localcache-0.0.0] startup
[2.335s] [ext: omni.kit.stage_templates-1.1.2] startup
[2.338s] [ext: omni.convexdecomposition-1.4.15] startup
[2.344s] [ext: omni.physics.tensors-0.1.0] startup
[2.358s] [ext: omni.physx-1.4.15-5.1] startup
[2.417s] [ext: omni.graph.core-2.29.1] startup
[2.445s] [ext: omni.kit.menu.utils-1.2.11] startup
[2.476s] [ext: omni.physx.tensors-0.1.0] startup
[2.492s] [ext: omni.graph-1.23.0] startup
[2.597s] [ext: omni.kit.numpy.commo

In [6]:
# set log level
import logging
import carb

logging.getLogger("omni.hydra").setLevel(logging.ERROR)
logging.getLogger("omni.isaac.urdf").setLevel(logging.ERROR)
logging.getLogger("omni.physx.plugin").setLevel(logging.ERROR)

logging.getLogger("omni.isaac.synthetic_utils").setLevel(logging.ERROR)
logging.getLogger("omni.isaac.synthetic_utils.syntheticdata").setLevel(logging.ERROR)
logging.getLogger("omni.hydra.scene_delegate.plugin").setLevel(logging.ERROR)


l = carb.logging.LEVEL_ERROR
carb.settings.get_settings().set("/log/level", l)
carb.settings.get_settings().set("/log/fileLogLevel", l)
carb.settings.get_settings().set("/log/outputStreamLevel", l)

# # This logged error is printed as it should
# carb.log_error("ERROR")
# # This warning is printed but should not
# carb.log_warn("WARNING")

# Config

In [7]:
from task.config import DATA_PATH, FEATURE_PATH
task_type = "Desk"
side_choice = "Border"
base_asset_id = 0
load_nucleus = True

In [8]:
pause

NameError: name 'pause' is not defined

# Init Env

In [9]:
from uva_env import UvaEnv
env = UvaEnv()

In [10]:
from task.utils import add_scene_default
add_scene_default()

In [11]:
print(list(env.stage.TraverseAll()))

[Usd.Prim(</Render>), Usd.Prim(</Render/RenderProduct_Viewport>), Usd.Prim(</Render/Vars>), Usd.Prim(</Render/Vars/LdrColor>), Usd.Prim(</physicsScene>), Usd.Prim(</World>), Usd.Prim(</World/defaultLight>), Usd.Prim(</OmniverseKit_Persp>), Usd.Prim(</OmniverseKit_Front>), Usd.Prim(</OmniverseKit_Top>), Usd.Prim(</OmniverseKit_Right>)]


In [12]:
env.clean()
env.world.step(render=True)

# Scene

In [13]:
from task.scene import ArrangeScene
scene = ArrangeScene(task_type, side_choice, base_asset_id = 0, traj_id = 0, load_nucleus = load_nucleus)
env.scene = scene

In [14]:
# add base
scene.add_base_asset()

asset_prim: omniverse://localhost/Users/yizhou/Asset/D/Desk/Desk_01.usd


In [15]:
# add room
scene.add_room()

asset_prim: omniverse://localhost/Users/yizhou/Asset/S/0/layout.usd


In [16]:
env.world.step(render=True)

In [17]:
# randomize scene
from layout.randomizer import Randomizer
randomizer = Randomizer(load_nucleus = load_nucleus)

# Reward

In [18]:
from uv.reward import Rewarder

In [19]:
rewarder = Rewarder(env.world)
env.rewarder = rewarder

# Render

In [20]:
from render.helper import RenderHelper
render = RenderHelper(task_type, side_choice)

render.add_task_cameras(camera_type="side")
render.add_task_cameras(camera_type="side")
render.set_cameras()

In [21]:
render.camera_paths

['/World/render/camera_side', '/World/render/camera_side']

In [22]:
pause

NameError: name 'pause' is not defined

#  Learning

In [23]:
# from learning.network.resnet import ResNetFeatureExtractor

# feature extraction
from learning.utils import extract_image_clip_feature_and_save, obtain_action_from_trainer
from transformers import CLIPProcessor, CLIPModel

feature_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
feature_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# replay buffer
import json
from learning.replay_buffer import ReplayBuffer
from learning.config import *

buffer = ReplayBuffer(max_size=2000)

# trainer
from learning.network.sac import *

policy = Policy()

qf1 = QFunction()
qf2 = QFunction()
target_qf1 = QFunction()
target_qf2 = QFunction()

from learning.sac_trainer import SACTrainer

trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
     use_automatic_entropy_tuning = False, 
     policy_lr=1e-3, 
     qf_lr=1e-3,
     target_update_period = 5)

# Trajectory

In [24]:
use_network = True
debug = False

In [25]:
total_traj = 0
total_step = 0

---------------------

In [27]:
# traj config
for traj_id in range(1, 100):
    print("total_traj: ", total_traj)
    env.scene.traj_id = traj_id
    
    # base
    if total_traj % 5 == 0:
        asset_count = len(env.scene.base_asset_file_paths)
        base_asset_id = random.choice([_ for _ in range(asset_count)])
        env.scene.base_asset_id = base_asset_id    
        env.clean(clean_base = True, clean_all = False)
        scene.add_base_asset()
        
        # randomize scene
        randomizer.randomize_house(rand = True)
        
    image_folder = os.path.join(DATA_PATH, task_type, side_choice, str(traj_id))
    
    env.world.step(render = True)
    env.world.step(render = True)

    # get images
    env.world.render()
    images = render.get_images()
    render.save_rgb(images[0]['rgb'], image_folder, "0")
    
    ## extract feature
    
#     extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
#         feature_extractor, os.path.join(image_folder, str(0) + ".pt"))
    if use_network:
        extract_image_clip_feature_and_save(f"{image_folder}/{0}.png", feature_model, feature_processor, 
            f"{image_folder}/{0}.pt",)
    

    # trajectory
    for step in range(5):
        total_step += 1
        
        # sample an object
        env.add_scene_obj(mode = "random")
        
        # TODO: get action from sampling
        if not use_network or total_traj < 10 or np.random.rand() < 0.2:
            x, y = np.tanh(np.random.randn()), np.tanh(np.random.randn())
        else:
            image_feature_file = f"{image_folder}/{step}.pt"
            
            object_info = env.scene.objects[-1]
            object_type = object_info["type"]
            obj_name = object_info["name"][:-4]
            object_feature_file = os.path.join(FEATURE_PATH, object_type, obj_name + ".pt")
            x, y = obtain_action_from_trainer(image_feature_file, object_feature_file, trainer, 
                                              scaler=np.exp(- total_traj / 100))
            object_info["use_network"] = True
        
        # load the object into the scene
        env.put_last_object((x, y)) 
        env.world.step(render=True)
        
        # register the object to the world for physics update
        env.register_last_object()
        env.world.step(render=True)

        # get images
        env.world.render()
        images = render.get_images()
        render.save_rgb(images[0]['rgb'], image_folder, str(step + 1))

        ## calculate reward
        env.calculate_last_reward(simulation_step = 30)
        
        ## extract feature
#         extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
#             feature_extractor, os.path.join(image_folder, str(step + 1) + ".pt"))
        if use_network:
            extract_image_clip_feature_and_save(f"{image_folder}/{step + 1}.png", feature_model, feature_processor, 
                f"{image_folder}/{step + 1}.pt",)
    

        ## reset
        env.world.reset()
        env.world.step(render=True)
        
        ## trainer nework
        if use_network and total_step % UPDATE_TRAINER_STEPS == 0 and total_traj > 5:
            batch = buffer.sample_batch(batch_size = BATCH_SIZE)
            trainer.update(batch)
            
            if debug and total_step % 10 == 0:
                rewards = batch['rewards'].to(device)
                terminals = batch['terminals'].to(device)
                obs = batch['observations'].to(device)
                actions = batch['actions'].to(device)
                next_obs = batch['next_observations'].to(device)
                obj_features = batch['object_features'].to(device)
                
                dist = trainer.policy(obs, obj_features)
                pred = trainer.qf1(obs, obj_features, actions)
                print(f"debug {total_traj}/{total_step}", #"\n dist: ", dist.mean.flatten().tolist(), dist.stddev.flatten().tolist(),
                      "\n pred:", pred.flatten().tolist(),
                      "\n rewards: ", rewards.flatten().tolist())


    # Record
    record = env.scene.get_scene_data()
    env.scene.save_scene_data()
    # print("record: ", record)
    
    # Add record to buffer
    if use_network:
        buffer.add_scene_sample(record)

    # Reset (env clean)
    env.clean(clean_all = False)
    env.step(render = True)
    total_traj += 1

total_traj:  1
2023-01-27 19:40:01 [120,500ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-27 19:40:01 [120,604ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Book/OV_CreateBook_03.usd
2023-01-27 19:40:02 [121,322ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Book

2023-01-27 19:40:02 [121,549ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-27 19:40:02 [121,653ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/DeskDecor/HippoOrganizer.usd
2023-01-27 19:40:05 [124,014ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to fals

2023-01-27 19:40:19 [138,277ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine

2023-01-27 19:40:19 [138,277ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Plant

2023-01-27 19:40:19 [138,277ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine

2023-01-27 19:40:19 [138,277ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Plant

2023-01-27 19:40:19 [138,277ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine

2023-01-27 19:40:19 [138,277ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Plant

2023-01-27 19:40:19 [138,277ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine

2023-01-27 19:4

total_traj:  3
2023-01-27 19:40:21 [140,265ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-27 19:40:21 [140,369ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Clock_Desk/AlarmClock_Retro.usd
2023-01-27 19:40:22 [141,111ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Clock_Desk

2023-01-27 19:40:22 [141,347ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-27 19:40:22 [141,450ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-27 19:40:23 [142,162ms] [Warning] [omni.usd] Coding Error: in AddXformOp at line 207 of /buildAgent/work/ca6c508eae419cf8/USD/pxr/usd/usdGeom/xformable.cpp -- XformOp </World/obje

2023-01-27 19:41:33 [212,605ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/DeskDecor

2023-01-27 19:41:33 [212,605ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/DeskDecor

2023-01-27 19:41:33 [212,605ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/DeskDecor

2023-01-27 19:41:33 [212,605ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/DeskDecor

2023-01-27 19:41:33 [212,605ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/DeskDecor

2023-01-27 19:41:33 [212,605ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/DeskDecor

2023-01-27 19:41:33 [212,605ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/DeskDecor


total_traj:  5
asset_prim: omniverse://localhost/Users/yizhou/Asset/D/Desk/Desk_01.usd
2023-01-27 19:41:37 [216,360ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-27 19:41:37 [216,464ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Book/Book_15.usd
2023-01-27 19:41:38 [217,050ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-27 19:41:38 [217,154ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-27 19:41:39 [217,802ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-27 19:41:39 [217,802ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2

...

total_traj:  99
2023-01-27 19:57:13 [1,152,564ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-27 19:57:13 [1,152,667ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Book/Book_15.usd
2023-01-27 19:57:14 [1,153,755ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-27 19:57:15 [1,153,857ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-27 19:57:15 [1,154,741ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-27 19:57:15 [1,154,741ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-27 19:57:15 [1,154,741ms] [Warning] [omni.hydra.scen

asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Sculpture/DecorativeDisk_14.usd
2023-01-27 19:57:17 [1,156,712ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Sculpture

2023-01-27 19:57:18 [1,157,181ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-27 19:57:18 [1,157,285ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-27 19:57:19 [1,158,174ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-27 19:57:19 [1,158,174ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Sculpture

2023-01-27 19:57:19 [1,158,174ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/TableLamp

2023-01-27 19:57:19 [1,158,174ms] [Warning] [omni

asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Plant/Plant_Succulent_01.usd
2023-01-27 19:57:19 [1,158,490ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Plant

2023-01-27 19:57:20 [1,159,343ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-27 19:57:20 [1,159,444ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-27 19:57:21 [1,160,314ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-27 19:57:21 [1,160,314ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Plant

2023-01-27 19:57:21 [1,160,314ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Sculpture

2023-01-27 19:57:21 [1,160,314ms] [Warning] [omni.hydra.scen

asset_prim: omniverse://localhost/Users/yizhou/Asset/I/DeskDecor/HippoOrganizer.usd
2023-01-27 19:57:21 [1,160,677ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/DeskDecor

2023-01-27 19:57:23 [1,162,037ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-27 19:57:23 [1,162,139ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-27 19:57:24 [1,163,027ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-27 19:57:24 [1,163,027ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/DeskDecor

2023-01-27 19:57:24 [1,163,027ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Plant

2023-01-27 19:57:24 [1,163,027ms] [Warning] [omni.hydra.

2023-01-27 19:57:24 [1,163,030ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/TableLamp

2023-01-27 19:57:24 [1,163,030ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/DeskDecor

2023-01-27 19:57:24 [1,163,030ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Sculpture

2023-01-27 19:57:24 [1,163,030ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/TableLamp

2023-01-27 19:57:24 [1,163,030ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/DeskDecor

2023-01-27 19:57:24 [1,163,030ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Sculpture

2023-01-27 19:57:24 [1,163,030ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/obje

# Debug

In [ ]:
env.clean(clean_base = True, clean_all = False)

In [ ]:
# get images
env.world.render()
env.world.render()
images = render.get_images()

Image.fromarray(images[0]['rgb'], "RGBA").resize((300, 300))

##  Debug Training

In [ ]:
import json
from learning.replay_buffer import ReplayBuffer

buffer = ReplayBuffer(max_size=1000)

for i in range(100):
    replay = json.load(open(f"{DATA_PATH}/{task_type}/{side_choice}/{i}/scene.json"))
    buffer.add_scene_sample(replay)

In [ ]:
len(buffer.dataset)

In [ ]:
# trainer
from learning.network.sac import *

policy = Policy()

qf1 = QFunction()
qf2 = QFunction()

target_qf1 = QFunction()
target_qf1.eval()

target_qf2 = QFunction()
target_qf2.eval()

from learning.sac_trainer import SACTrainer

trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
     use_automatic_entropy_tuning = True, 
     policy_lr=1e-3, 
     qf_lr=1e-3,
     target_update_period = 5)

In [ ]:
self = trainer

In [ ]:
for _ in range(500):
    batch = buffer.sample_batch(batch_size=6)

    rewards = batch['rewards'].to(device)
    terminals = batch['terminals'].to(device)
    obs = batch['observations'].to(device)
    actions = batch['actions'].to(device)
    next_obs = batch['next_observations'].to(device)

    obj_features = batch['object_features'].to(device)

    # print("rewards", rewards)

    self.update(batch)
    
    if _ % 10 == 0:
        dist = self.policy(obs, obj_features)
        pred = self.qf1(obs, obj_features, actions)
        print(_, "\n dist   : ", [round(x, 3) for x in dist.mean.flatten().tolist()], 
                  [round(x, 3) for x in dist.stddev.flatten().tolist()],
              "\n pred   :", [round(x, 3) for x in pred.flatten().tolist()],
              "\n rewards: ", [round(x, 3) for x in rewards.flatten().tolist()])
    
#     # debug
#     pred = self.qf1(obs, obj_features, actions)
#     loss = self.qf_criterion(pred, rewards)
    
#     self.qf1_optimizer.zero_grad()
#     loss.backward()
#     self.qf1_optimizer.step()
    
#     if _ % 20 == 0:
#         print(_, "\n debug loss: ", loss.item(), "\n pred:", 
#               pred.flatten().tolist(), "\n rewards: ", rewards.flatten().tolist())

In [ ]:
dist = self.policy(obs.to(device), obj_features.to(device))

In [ ]:
dist.normal_mean, dist.stddev

In [ ]:
dist.sample(scaler = 0.2)

In [ ]:
self = dist

In [ ]:
scaler = 0.1

In [ ]:
self.normal_mean + scaler * self.normal_std * MultivariateDiagonalNormal(
    torch.zeros(self.normal_mean.size()).to(self.device),
    torch.ones(self.normal_std.size()).to(self.device)
).sample()

# distribution

In [ ]:
from learning.distributions import MultivariateDiagonalNormal

In [ ]:
dist = JustNormal(dist.mean, dist.stddev)

In [ ]:
dist.rsample_and_logprob()